In [14]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Load the dataset
file_path = 'data.csv'  # Update this path to your actual file
data = pd.read_csv(file_path)

# Select relevant features for similarity calculations
features = data[['valence', 'acousticness', 'danceability', 'energy',
                 'instrumentalness', 'liveness', 'speechiness', 'tempo']]

# Normalize features for better similarity calculations
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Initialize the nearest neighbors model
neighbors_model = NearestNeighbors(metric='cosine', algorithm='brute')
neighbors_model.fit(scaled_features)

# Function to find song index by name
def find_song_index(song_name):
    # Search for the song by name
    song = data[data['name'].str.contains(song_name, case=False, na=False)]
    if song.empty:
        print("Song not found.")
        return None
    return song.index[0]

# Function to recommend songs based on a song name input
def recommend_songs_by_name(song_name, num_recommendations=5):
    song_index = find_song_index(song_name)
    if song_index is None:
        return None
    # Reshape the target song's features for querying
    song_features = scaled_features[song_index].reshape(1, -1)
    # Find the nearest neighbors (similar songs)
    distances, indices = neighbors_model.kneighbors(song_features, n_neighbors=num_recommendations + 1)
    # Exclude the song itself and format recommendations
    recommended_indices = indices.flatten()[1:]  # Exclude first (same song)
    recommended_songs = data.iloc[recommended_indices]
    return recommended_songs[['name', 'artists', 'year', 'popularity']]

# Prompt user to enter a song name
user_input = input("Enter a song name to get recommendations: ")
recommended_songs = recommend_songs_by_name(user_input)

# Display recommended songs
if recommended_songs is not None:
    print(recommended_songs)



Enter a song name to get recommendations: waka waka
                                       name                    artists  year  \
166111                               Salomé               ['Chayanne']  1998   
169824                              Worship                  ['Lizzo']  2016   
70704                                Salomé               ['Chayanne']  1998   
31450   Out In L.A. - 2002 Digital Remaster  ['Red Hot Chili Peppers']  1984   
149079                          Canned Heat             ['Jamiroquai']  1987   

        popularity  
166111          48  
169824          53  
70704           57  
31450           52  
149079          32  
